## CESM2 - LARGE ENSEMBLE (LENS2)

#### by Mauricio Rocha and Dr. Gustavo Marques

- This notebooks servers as an example on how to extract surface (or any other 2D spatial field) properties from a selected spacial region accross all LENS2 members for the ocean component for the SSP370. 

## Imports

In [ ]:
import intake
import intake_esm
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import fsspec
import cmocean
import cartopy
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pop_tools
import sys
from distributed import Client
from ncar_jobqueue import NCARCluster
sys.path.append('../functions')
import util
from cartopy.util import add_cyclic_point
from misc import get_ij
import warnings, getpass, os

<div class="alert alert-block alert-info">
<b>Note:</b> comment the following line when debugging
</div>

In [ ]:
warnings.filterwarnings("ignore")

### Local functions

In [ ]:
def rms_da(da, dims=('nlat', 'nlon'), weights=None,  weights_sum=None):
  """
  Calculates the rms in DataArray da (optional weighted rms).

  ----------
  da : xarray.DataArray
        DataArray for which to compute (weighted) rms.

  dims : tuple, str
    Dimension(s) over which to apply reduction. Default is ('yh', 'xh').

  weights : xarray.DataArray, optional
    weights to apply. It can be a masked array.

  weights_sum : xarray.DataArray, optional
    Total weight (i.e., weights.sum()). Only computed if not provided.

  Returns
  -------
  reduction : DataSet
      xarray.Dataset with (optionally weighted) rms for da.
  """

  if weights is not None:
    if weights_sum is None: weights_sum = weights.sum(dim=dims)
    out = np.sqrt((da**2 * weights).sum(dim=dims)/weights_sum)
    # copy attrs
    out.attrs = da.attrs
    return out
  else:
    return np.sqrt((da**2).mean(dim=dims, keep_attrs=True))

### Dask workers

In [ ]:
mem_per_worker = 140 # in GB 
num_workers = 80 
cluster = NCARCluster(cores=4, processes=3, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=6:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Data Ingest

In [ ]:
%%time
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

In [ ]:
catalog.search(component='ocn').unique('frequency')

### Let's search for variables with montly frequency

In [ ]:
cat_subset = catalog.search(component='ocn',
                            frequency='month_1',
                            variable=['TEMP','SHF','XMXL']) # TEMP, Total Surface Heat Flux, Maximum Mixed Layer Depth

In [ ]:
%%time
dset_dict_raw = cat_subset.to_dataset_dict()

In [ ]:
# print keys
[key for key in dset_dict_raw.keys()]

In [ ]:
# Variables
fb=('TEMP','SHF','XMXL')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'ds_ssp370_cmip6_{fb[ifb]} = dset_dict_raw[\'ocn.ssp370.pop.h.cmip6.{fb[ifb]}\']'
    exec(str)
    str=f'ds_ssp370_smbb_{fb[ifb]} = dset_dict_raw[\'ocn.ssp370.pop.h.smbb.{fb[ifb]}\']'
    exec(str)
    str=f'ds_ssp370_{fb[ifb]} = xr.concat([ds_ssp370_cmip6_{fb[ifb]},ds_ssp370_smbb_{fb[ifb]}], dim=\'member_id\',data_vars=\'minimal\',coords=\'minimal\',compat=\'override\')'
    exec(str)
    str=f'mem=ds_ssp370_{fb[ifb]}.{fb[ifb]}.nbytes*1e-12 # in TB'
    exec(str)
    print(f'Memory: {mem} TB')
    str=f'del ds_ssp370_cmip6_{fb[ifb]}; del ds_ssp370_smbb_{fb[ifb]}'; exec(str) 
print(f'Done!')

### Import the POP grid

If you choose the ocean component of LENS2, you will need to import the POP grid. For the other components, you can use the emsemble's own grid. 

In ds, TLONG and TLAT have missing values (NaNs), so we need to override them with the values from pop_grid, which does not have missing values.

In [ ]:
# Read the pop 1 deg grid from pop_tools
# We will use variables TLONG and TLAT
pop_grid = pop_tools.get_grid('POP_gx1v7')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'ds_ssp370_{fb[ifb]}[\'TLONG\'] = pop_grid.TLONG'     # Longitud
    exec(str)
    str=f'ds_ssp370_{fb[ifb]}[\'TLAT\'] = pop_grid.TLAT'       # Latitudes
    exec(str)
    str=f'ds_ssp370_{fb[ifb]}[\'TLONG\'] = pop_grid.TLONG'     # Longitud
    exec(str)
    str=f'ds_ssp370_{fb[ifb]}[\'TLAT\'] = pop_grid.TLAT'       # Latitudes
    exec(str)

In [ ]:
ds_var_1231 = ds_ssp370_XMXL.XMXL.sel(member_id=['r1i1231p1f1','r2i1231p1f1','r3i1231p1f1','r4i1231p1f1','r5i1231p1f1','r6i1231p1f1','r7i1231p1f1','r8i1231p1f1','r9i1231p1f1','r10i1231p1f1']).mean(dim='time')#.plot()
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = ds_var_1231.mean(dim='member_id').plot.pcolormesh(ax=ax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmocean.cm.balance,
                    x='TLONG',
                    y='TLAT',
                    #vmin=-3,
                    #vmax=30,
                    cbar_kwargs={'orientation': 'horizontal'})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines();
plt.title('i1231p1f1')
del ds_var_1231

### Centralize the South Atlantic 
Need to combine the domain in the east/west direction to centralize the South Atlantic

In [ ]:
ilat, flat = 101, 182
ilon1, flon1, ilon2, flon2 = 313, 320, 0, 57
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'sa_ds_{fb[ifb]}=xr.combine_nested([[ds_ssp370_{fb[ifb]}.isel(nlat = slice(ilat,flat),nlon = slice(ilon1,flon1)),ds_ssp370_{fb[ifb]}.isel(nlat = slice(ilat,flat),nlon = slice(ilon2,flon2))]],concat_dim=[\'nlat\',\'nlon\'])'
    exec(str)
    str=f'sa_ds_{fb[ifb]}.coords[\'TLONG\'] = (sa_ds_{fb[ifb]}.coords[\'TLONG\'] + 180) % 360 - 180' # change the longitudes: -180 0 180
    exec(str)
    str=f'del ds_ssp370_{fb[ifb]}'; exec(str)

In [ ]:
# simple check
sa_ds_XMXL.XMXL.isel(time=2, member_id=0).plot()

In [ ]:
%%time
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = sa_ds_XMXL.XMXL.isel(member_id=0).mean(dim='time').plot.pcolormesh(ax=ax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmocean.cm.balance,
                    x='TLONG',
                    y='TLAT',
                    #vmin=10,
                    #vmax=30,
                    cbar_kwargs={"orientation": "horizontal"})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines();

### Extract correponding area 

In [ ]:
area_sa = xr.combine_nested([
    [pop_grid.TAREA.isel(nlat = slice(ilat,flat),nlon = slice(ilon1,flon1)),
     pop_grid.TAREA.isel(nlat = slice(ilat,flat),nlon = slice(ilon2,flon2))]],
    concat_dim=['nlat','nlon']
)

In [ ]:
# simple check
area_sa.plot();

### Calculate anomalies
Let's calculate the heat stored from the temperature anomaly field (data minus the average temperature for the whole time series)

In [ ]:
# Temperature in degress Kelvin
sa_ds_TEMP_K=sa_ds_TEMP.TEMP+273.15
sa_ds_TEMP_K.coords['z_t']=sa_ds_TEMP_K.coords['z_t']*0.01 # cm to m

In [ ]:
sa_ds_TEMP_K_anom=sa_ds_TEMP_K-(sa_ds_TEMP_K.mean('time'))

### Calculte the heat storage:

#### The formula for this is: $$\rm{HS = \uprho_\uptheta~C_p~\int_{z_2}^{z_1}\uptheta_{(z)}'~dz},$$
where:
* HS is heat storage ($\rm{J~m^{-2}}$),
* $\uprho$ is the density of sea water, 1026 $\rm{kg~m^{-3}}$,
* $\rm{C_p}$ is the specific heat of sea water, 3996 $\rm{J~kg^{-1}~K^{-1}}$, do modelo
* $\rm{z}$ is the depth limit o the calculation in meters,
* and $\uptheta$' is the potential temperature monthly anomaly at each depth in degress Kelvin. 

In [ ]:
sa_ds_dz=sa_ds_TEMP.dz*0.01 # cm to m
sa_ds_dz.coords['z_t']=sa_ds_dz.coords['z_t']*0.01 # cm to m

<div class="alert alert-block alert-info">
<b>Note:</b> Although most of the variation in heat storage occurs in the first 1000 meters of depth, we will try to add the depth of the AMOC core to this calculation to integrate the temperature up to 1573 m. Ideally, we would compute the heat content in the upper branch of the AMOC, i.e., not use a fixed depth.

In [ ]:
sa_ds_TEMP_K_anom.coords['z_t'][51]

In [ ]:
rho = 1026 #kg/m^3
c_p = 3996 #J/(kg K) - I used the same amount of specific heat used by the model
sa_ds_HS=(sa_ds_TEMP_K_anom*sa_ds_dz).sel(z_t=slice(0,43)).sum(dim='z_t')*rho*c_p
sa_ds_HS=sa_ds_HS.where(sa_ds_HS != 0.) # The continents was equal to zero. We replaced zero with NaN, because the ocean heat content will never equal zero. 
sa_ds_HS.isel(member_id=0,time=0).plot()
del sa_ds_TEMP_K; del sa_ds_TEMP_K_anom 

In [ ]:
sa_ds_HS= xr.merge([sa_ds_HS.rename('HS')])

In [ ]:
# Annual Mean
sa_ds_HS=sa_ds_HS.resample(time='1Y', closed='left').mean('time')
#sa_ds_TEMP=sa_ds_TEMP.resample(time='1Y', closed='left').mean('time')
sa_ds_SHF=sa_ds_SHF.resample(time='1Y', closed='left').mean('time')
#sa_ds_XMXL=sa_ds_XMXL.resample(time='1Y', closed='left').mean('time')

In [ ]:
#del sa_ds_dz; del sa_ds_TEMP; del sa_ds_XMXL

In [ ]:
#sa_ds_HS

### Perfom computations
Calculate area mean, min, max, and rms for the surface temperature of the selected region

In [ ]:
%%time
start="2015-02-01"
end="2100-12-31"
fb=('HS','SHF')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'var_{fb[ifb]} = sa_ds_{fb[ifb]}.{fb[ifb]}.sel(time=slice(start,end))'
    exec(str)
    print(f'var_{fb[ifb]}')
    # Mean
    str=f'var_mean_{fb[ifb]} = var_{fb[ifb]}.weighted(area_sa).mean(dim=(\'nlon\',\'nlat\')).load()'
    exec(str)
    print(f'var_mean_{fb[ifb]}')
    # Maximum
    str=f'var_max_{fb[ifb]} = var_{fb[ifb]}.max(dim=(\'nlon\',\'nlat\')).load()'
    exec(str)
    print(f'var_max_{fb[ifb]}')
    # Minimum
    str=f'var_min_{fb[ifb]} = var_{fb[ifb]}.min(dim=(\'nlon\',\'nlat\')).load()'
    exec(str)
    print(f'var_min_{fb[ifb]}')
    # RMS
    str=f'var_rms_{fb[ifb]} = rms_da(var_{fb[ifb]}, weights=area_sa, weights_sum=area_sa.sum()).load()'
    exec(str)
    print(f'var_rms_{fb[ifb]}')

### TODO
Plot some time series to check calculations

### Merge data and save on disk

In [ ]:
#units=('J/m2','W/m2','cm','oC')
#fb=('HS','SHF','XMXL','TEMP')
units=('J/m2','W/m2')
#long_name=('Total Surface Heat Flux','Maximum Mixed Layer Depth','Ocean Heat Content')
long_name=('Heat Storage','Total Surface Heat Flux')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'ds_out_{fb[ifb]} = xr.merge([var_rms_{fb[ifb]}.rename(\'{fb[ifb]}_rms\'),var_mean_{fb[ifb]}.rename(\'{fb[ifb]}_mean\'),var_max_{fb[ifb]}.rename(\'{fb[ifb]}_max\'),var_min_{fb[ifb]}.rename(\'{fb[ifb]}_min\')])'
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'description\'] = \'{long_name[ifb]} ({fb[ifb]}) statistics for the South Atlantic (47.5W-23.75E and 25.91623S-1.413613S)\''
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'units\'] = \'{units[ifb]}\''
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'author\'] = \'Mauricio Rocha\''
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'email\'] = \'mauricio.rocha@usp.br\''
    exec(str)

In [ ]:
# Total Surface Heat Fux
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_SHF.SHF_max.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
ds_out_SHF.SHF_max.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
ds_out_SHF.SHF_max.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('SHF Max [W/m2]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
#axes[0].set_ylim(103,108.2)
fig.tight_layout(pad=2.0)
# Mean
ds_out_SHF.SHF_mean.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
ds_out_SHF.SHF_mean.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
ds_out_SHF.SHF_mean.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('SHF Mean [W/m2]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()
#axes[1].set_ylim(17,25.5)
# Minimum
ds_out_SHF.SHF_min.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
ds_out_SHF.SHF_min.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
ds_out_SHF.SHF_min.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('SHF Min [W/m2]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()
#axes[2].set_ylim(-63,-51)
# Minimum
ds_out_SHF.SHF_rms.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
ds_out_SHF.SHF_rms.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
ds_out_SHF.SHF_rms.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('SHF Error [W/m2]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()
#axes[3].set_ylim(50.3,52)

plt.show()

In [ ]:
# Temperature
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_TEMP.TEMP_max.isel(z_t=0).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_max.isel(z_t=0).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y -5m')
ds_out_TEMP.TEMP_max.isel(z_t=0).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y -5m')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('TEMP Max [oC]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
#axes[0].set_ylim(28.78,30.015)
fig.tight_layout(pad=2.0)
# Mean
ds_out_TEMP.TEMP_mean.isel(z_t=0).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_mean.isel(z_t=0).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y -5m')
ds_out_TEMP.TEMP_mean.isel(z_t=0).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y -5m')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('TEMP Mean [oC]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()
#axes[1].set_ylim(25.7,26.7)
# Minimum
ds_out_TEMP.TEMP_min.isel(z_t=0).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_min.isel(z_t=0).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y -5m')
ds_out_TEMP.TEMP_min.isel(z_t=0).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y -5m')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('TEMP Min [oC]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()
#axes[2].set_ylim(20.7,21.7)
# Minimum
ds_out_TEMP.TEMP_rms.isel(z_t=0).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_rms.isel(z_t=0).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y -5m')
ds_out_TEMP.TEMP_rms.isel(z_t=0).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y -5m')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('TEMP Error [oC]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()
#axes[3].set_ylim(19.4,20.2)

plt.show()

In [ ]:
# Temperature
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_TEMP.TEMP_max.isel(z_t=40).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.1,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_max.isel(z_t=40).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",alpha=0.6,color='r',linewidth=1,label='Member Mean 1Y -1106m')
ds_out_TEMP.TEMP_max.isel(z_t=40).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y -1106m')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('TEMP Max [oC]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
#axes[0].set_ylim(28.78,30.015)
fig.tight_layout(pad=2.0)
# Mean
ds_out_TEMP.TEMP_mean.isel(z_t=40).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.1,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_mean.isel(z_t=40).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",alpha=0.6,color='k',linewidth=1,label='Member Mean 1Y -1106m')
ds_out_TEMP.TEMP_mean.isel(z_t=40).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y -1106m')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('TEMP Mean [oC]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()
#axes[1].set_ylim(25.7,26.7)
# Minimum
ds_out_TEMP.TEMP_min.isel(z_t=40).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.1,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_min.isel(z_t=40).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",alpha=0.6,color='b',linewidth=1,label='Member Mean 1Y -1106m')
ds_out_TEMP.TEMP_min.isel(z_t=40).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y -1106m')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('TEMP Min [oC]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()
#axes[2].set_ylim(20.7,21.7)
# Minimum
ds_out_TEMP.TEMP_rms.isel(z_t=40).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.1,linewidth=1,add_legend=False)
ds_out_TEMP.TEMP_rms.isel(z_t=40).resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",alpha=0.6,color='g',linewidth=1,label='Member Mean 1Y -1106m')
ds_out_TEMP.TEMP_rms.isel(z_t=40).resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y -1106m')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('TEMP Error [oC]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()
#axes[3].set_ylim(19.4,20.2)

plt.show()

In [ ]:
# Maximum mixed layer depth
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
(ds_out_XMXL.XMXL_max.resample(time='1Y', closed='left').mean('time')*-0.01).plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_XMXL.XMXL_max.resample(time='1Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
(ds_out_XMXL.XMXL_max.resample(time='5Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('XMXL Max [m]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
#axes[0].set_ylim(103,108.2)
fig.tight_layout(pad=2.5)
# Mean
(ds_out_XMXL.XMXL_mean.resample(time='1Y', closed='left').mean('time')*-0.01).plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_XMXL.XMXL_mean.resample(time='1Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
(ds_out_XMXL.XMXL_mean.resample(time='5Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('XMXL Mean [m]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()
#axes[1].set_ylim(17,25.5)
# Minimum
(ds_out_XMXL.XMXL_min.resample(time='1Y', closed='left').mean('time')*-0.01).plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_XMXL.XMXL_min.resample(time='1Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
(ds_out_XMXL.XMXL_min.resample(time='5Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('XMXL Min [m]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()
#axes[2].set_ylim(-63,-51)
# Minimum
(ds_out_XMXL.XMXL_rms.resample(time='1Y', closed='left').mean('time')*-0.01).plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_XMXL.XMXL_rms.resample(time='1Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
(ds_out_XMXL.XMXL_rms.resample(time='5Y', closed='left').mean('time').mean('member_id')*-0.01).plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('XMXL Error [m]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()
#axes[3].set_ylim(50.3,52)

plt.show()

In [ ]:
# Heat Strage
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_HS.HS_max.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
ds_out_HS.HS_max.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y 0-1573m')
ds_out_HS.HS_max.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y 0-1573m')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('OHC Max [J/m2]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
#axes[0].set_ylim(103,108.2)
fig.tight_layout(pad=2.5)
# Mean
ds_out_HS.HS_mean.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
ds_out_HS.HS_mean.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y 0-1573m')
ds_out_HS.HS_mean.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y 0-1573m')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('OHC Mean [J/m2]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()
#axes[1].set_ylim(17,25.5)
# Minimum
ds_out_HS.HS_min.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
ds_out_HS.HS_min.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y 0-1573m')
ds_out_HS.HS_min.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y 0-1573m')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('OHC Min [J/m2]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()
#axes[2].set_ylim(-63,-51)
# Minimum
ds_out_HS.HS_rms.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
ds_out_HS.HS_rms.resample(time='1Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y 0-1573m')
ds_out_HS.HS_rms.resample(time='5Y', closed='left').mean('time').mean('member_id').plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y 0-1573m')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('OHC Error [J/m2]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()
#axes[3].set_ylim(50.3,52)

plt.show()

### Let's save the data in netcdf format

In [ ]:
# create a directory on scratch to save the output
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'path = \'/glade/scratch/mauricio/LENS2_ssp370_south_atlantic/{fb[ifb]}/\'.format(getpass.getuser())'
    exec(str)
    str=f'os.system(\'mkdir -p \'+path)'
    exec(str)
    str=f'ds_out_{fb[ifb]}.to_netcdf(path+\'{fb[ifb]}_stats.nc\')'
    exec(str)

In [ ]:
cluster.close()
client.close()